## Run this to reset DB
```
MATCH (n)
DETACH DELETE n
```

## We imported the 4 main data sets here

In [ ]:
#bad practice will fix later
import dask_geopandas as dgpd
from neo4j import GraphDatabase
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pyproj import Transformer, CRS

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")
driver = GraphDatabase.driver(URI, auth=AUTH)

# the commented out queries are because the nodes are created but not connected so it adds the data to nothing

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    # read geojson in gpd
    pc4_muni_density = gpd.read_file("data/zh_pc4_gdf_with_municipality_and_ev_charger_density.geojson")
    # rename Government of Rotterdam to Rotterdam
    pc4_muni_density['municipality'] = pc4_muni_density['municipality'].replace(
        {"Government of Rotterdam": "Rotterdam"}
    )
    # display(pc4_muni_density)
    combined_charging_points = gpd.read_file("data/combined-charging-points-zuid-holland.geojson")
    # display(combined_charging_points)
    muni_translated = pd.read_json("data/municipalities_data_translated.json")
    # display(muni_translated)
    parking_candidates = gpd.read_file("data/candidates.gpkg")

    # get closest ev charger to each parking candidate
    def generate_closest_points(gdf_source, gdf_target):
        # Convert the GeoDataFrames to EPSG:3857 for accurate distance calculation
        transformer = Transformer.from_crs(CRS("EPSG:4326"), CRS("EPSG:3857"), always_xy=True)
        def find_nearest_point(point, gdf_target):
            # Find the nearest point in gdf_target to the given point
            nearest_geom = nearest_points(point, gdf_target.unary_union)[1]
            # Transform coordinates for distance calculation
            p1 = Point(transformer.transform(point.x, point.y))
            p2 = Point(transformer.transform(nearest_geom.x, nearest_geom.y))
            distance = p1.distance(p2)
            return nearest_geom.x, nearest_geom.y, distance
        gdf_source['nearest_lon'], gdf_source['nearest_lat'], gdf_source["distance_to_nearest"] = zip(
            *gdf_source.geometry.apply(lambda x: find_nearest_point(x, gdf_target))
        )
        return gdf_source
    
    parking_candidates = generate_closest_points(parking_candidates, combined_charging_points)

    # add pc4_muni_density to neo4j
    def add_pc4_muni_density(df):
        with driver.session() as session:
            for _, row in df.iterrows():
                query = """
                MERGE (m:Municipality {name: $municipality})
                MERGE (p:PC4Area {pc4_code: $pc4_code})
                SET p.density = $density, p.name = $pc4_code
                MERGE (p)-[:IS_LOCATED_IN]->(m)
                """
                session.run(query, 
                            municipality=row['municipality'], 
                            pc4_code=row['pc4_code'], 
                            density=row['density'])
    add_pc4_muni_density(pc4_muni_density)

    def add_combined_charging_points(df):
        with driver.session() as session:
            for _, row in df.iterrows():
                query = """
                MERGE (p:PC4Area {pc4_code: $pc4_code})

                MERGE (ev:EVChargingStation {
                    lat : $lat, 
                    lon : $lon, 
                    nearest_lat : $nearest_lat, 
                    nearest_lon : $nearest_lon, 
                    distance_to_nearest : $distance_to_nearest
                    }
                )

                MERGE (ev)-[:IS_LOCATED_IN]->(p)

                WITH p, ev
                MATCH (m:Municipality)<-[:IS_LOCATED_IN]-(p)
                MERGE (ev)-[:IS_LOCATED_IN]->(m)
                """
                session.run(query, 
                            pc4_code=row['pc4_code'], 
                            lat=row['lat'], 
                            lon=row['lon'], 
                            nearest_lat=row['nearest_lat'], 
                            nearest_lon=row['nearest_lon'], 
                            distance_to_nearest=row['distance_to_nearest'])
    add_combined_charging_points(combined_charging_points)

    def add_municipality_data(df):
        with driver.session() as session:
            for _, row in df.iterrows():
                query = """
                MERGE (m:Municipality {name: $municipality})
                SET m.home_value = $home_value,
                    m.households = $households,
                    m.vehicles = $vehicles,
                    m.passenger_cars = $passenger_cars,
                    m.population_density = $population_density
                """
                session.run(query, 
                            municipality=row['Municipality'], 
                            area=row['Area'],
                            addresses=row['Addresses'],
                            dwellinngs=row['Dwellings'],
                            population=row['Population'],
                            business_establishments=row['BusinessEstablishments'],
                            home_value=row['HomeValue'], 
                            households=row['Households'], 
                            vehicles=row['Vehicles'], 
                            passenger_cars=row['PassengerCars'], 
                            population_density=row['PopulationDensity'])
    add_municipality_data(muni_translated)
    
    def add_parking_candidates(df):
        with driver.session() as session:
            for _, row in df.iterrows():
                query = """
                MERGE (p:PC4Area {pc4_code: $pc4_code})
                MERGE (c:CandidateLocation {
                    lat: $lat, 
                    lon: $lon,
                    nearest_lat : $nearest_lat, 
                    nearest_lon : $nearest_lon, 
                    distance_to_nearest : $distance_to_nearest
                })
                MERGE (c)-[:IS_LOCATED_IN]->(p)

                WITH p, c
                MATCH (m:Municipality)<-[:IS_LOCATED_IN]-(p)
                MERGE (c)-[:IS_LOCATED_IN]->(m)
                """
                session.run(query, 
                            pc4_code=row['pc4_code'], 
                            lat=row['geometry'].y, 
                            lon=row['geometry'].x,
                            nearest_lat=row['nearest_lat'], 
                            nearest_lon=row['nearest_lon'], 
                            distance_to_nearest=row['distance_to_nearest'])
    add_parking_candidates(parking_candidates)

    
    driver.close()





/var/folders/zk/99phrd0d2fs5xjbc622wn4_h0000gn/T/ipykernel_18482/980874636.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  nearest_geom = nearest_points(point, gdf_target.unary_union)[1]
